In [ ]:
from pavlov import *
monitor(-1)

In [ ]:
from pavlov import stats
stats.plot(['*common-flight league-net-test', '*musty-ease*', '*meek-wools'], 'elo-mohex', 'μ', skip=1, fill=True)

In [ ]:
stats.plot(['*common-flight league-net-test', '*musty-ease*', '*meek-wools', -1], 'rel-entropy.policy', skip=1, fill=True)

In [ ]:
from pavlov import stats
stats.review(-1)

In [ ]:
from pavlov.stats import registry
sorted(registry.StatsReaders(-1))